# High numerical aperture, large field-of-view oblique plane microscopy

[Preprint](https://www.biorxiv.org/content/10.1101/2020.04.07.030569v2)  
[Github repo](https://github.com/qi2lab/OPM)  
Based on original Micromanager 2.0 beanshell script that performed the same acquisition steps. Moving to Python is helpful because we do not have to separately call our fluidics units through an external Python program.
  
October 2020  
Douglas Shepherd, PhD  
qi2lab  
Arizona State University  
douglas.shepherd@asu.edu

## Imports

In [22]:
from pycromanager import Bridge, Acquisition
from time import sleep
import numpy as np
from Pathlib import Path

## Define pycromanager specific hook functions for hardware controlled acquisition

### function to start stage
This is run once after the camera is put into active mode in the sequence acquisition. The stage starts moving on this command and outputs a TTL pulse to the camera when it passes the preset initial position. This TTL starts the camera running at the set exposure time using internal timing. The camera acts the master signal for the galvo/laser controller using its own "exposure out" signal.

In [11]:
def start_stage(event):
    stage_port = 'COM4'
    stage_message='1SCAN'
    send_command_Tiger(stage_port, stage_message, False)
    
    return event

## Define other hardware interaction functions

### function for error redundant communication with Tiger controller
Theoretically, this should not be necessary. But often, we get errors when communicating with our Tiger controller via the serial port. We also often see the polling function stop reporting the current stage positions in MM 2.0 GUI. This might be a computer specific issue. However, it is just easier to make sure the ASI Tiger received, acknowledges, and executes command as requested.  
  
This function also has the capability to continually poll the stage controller to check if a command is finished. We find this is necessary when moving the stage large distances.

In [ ]:
def send_command_Tiger(stage_port,command,check_busy):

    answer = ""
    send_success = False
    
    # send command
    while(~send_success):
        
        exception = False
        try:
            core.set_serial_port_command(stage_port, command, "\r")
        except ValueError:
            sleep(0.1)
            send_success = False
            exception = True
        if (~exception):
            send_success=True
        
        # receive response
        send_success = False
        exception = False
        try:
            answer = core.get_serial_port_answer(stage_port, "\r\n")
        except ValueError:
            sleep(0.1)
            send_success = False
            exception = True
        if (~exception): 
            send_success = True

    
    # if checkbusy is True, poll Tiger until stage is available
    if (check_busy):
        
        answer = 'X'
        
        while (~answer == 'N'):
            answer=""
            send_success = True
        
            # make sure command and response were received
            while (~send_success):
                # send command
                    exception = False
                try:
                    mmc.set_serial_port_command(stage_port, "STATUS", "\r")
                except ValueError: 
                    sleep(0.1)
                    send_success = False
                    exception = True
                if (~exception):
                    send_success = True

            # receive response
            send_success = False;
            exception = False;

            try:
                answer = mmc.get_serial_port_answer(stage_port, "\r\n")
            except ValueError:
                sleep(0.1)
                send_success = False
                exception = True
            if (~exception):
                send_success = True
            

    return answer

### function to set up galvo/laser controller

In [17]:
def setup_galvo(galvo_port,min_12bit_value,max_12bit_value,factor,channel,number_of_images):
    
    # construct command string
    galvo_command = 's_c'+str(channel)+'_l'+str(min_12bit_value)+'_h'+str(max_12bit_value)+'_f'+str(factor)+'_n'+str(number_of_images)+'_e'
    
    # send command string
    core.set_serial_port_command(galvo_port, message, '\r')
    
    # receive response
    
    return True

IndentationError: unexpected indent (<ipython-input-17-38cba513337b>, line 5)

## Create bridge to micromanager

In [2]:
bridge = Bridge()
core = bridge.get_core()

## Acquistion parameters

### Select laser channels and powers

In [ ]:
# lasers to use
# 0 -> inactive
# 1 -> active

state_405 = 0
state_488 = 0
state_561 = 1
state_635 = 0
state_730 = 0

# laser powers (0 -> 100%)

power_405 = 0
power_488 = 0
power_561 = 10
power_635 = 0
power_730 = 0

# construct boolean array for lasers to use
channel_states = [state_405,state_488,state_561,state_635,state_730]
channel_powers = [power_405,power_488,power_561,power_635,power_730]

### Camera parameters

In [10]:
# FOV parameters
ROI = [1152, 2048, 896, 0] #unit: pixels

# camera exposure
exposure_ms = 5 #unit: ms

# camera pixel size
pixel_size = .115 # unit: um

### Stage scan parameters

In [ ]:
# scan axis limits. Use stage positions reported by MM
scan_axis_start_asi = #unit: 1/10 um
scan_axis_end_asi = #unit: 1/10 um

# tile axis limits. Use stage positions reported by MM
tile_axis_start_asi = #unit: 1/10 um
tile_axis_end_asi = #unit: 1/10 um

# height axis limits. Use stage positions reported by MM
height_axis_start_asi = #unit: 1/10 um
height_axis_end_asi = #unit: 1/10 um

### Root directory

In [9]:
save_directory = Path('E:\data\20201005\')

## Setup hardware for stage scanning sample through oblique digitally scanned light sheet

### Calculate stage limits and speeds

In [4]:
# scan axis setup
scan_axis_step_um = 0.2  # unit: um
scan_axis_step_mm = step_size_um / 1000.
scan_axis_range_asi = np.abs(scan_axis_end_asi-scan_axis_start_asi)  # unit: 1/10 um
scan_axis_range_mm = scan_axis_range_asi / 10000 #unit: mm
exposure_s = exposure_ms / 1000. #unit: s
scan_axis_speed = np.round(scan_step_mm / exposure_s,2) #unit: mm/s
number_of_images = np.rint(scan_axis_range_mm / scan_axis_step_mm).astype(int)

# tile axis setup
tile_axis_overlap=0.2
tile_axis_range_asi = np.abs(tile_axis_end_asi - tile_axis_start_asi) #unit: 1/10 um
tile_axis_range_mm = tile_axis_range_asi / 10000 #unit: mm
tile_axis_ROI = ROI[3]-ROI[1] #unit: pixel
tile_step_um = (tile_axis_ROI*pixel_size_um) * (1-tile_axis_overlap) #unit: um
tile_step_mm = (tile_axis_ROI*pixel_size_um) * (1-tile_axis_overlap) * .001 #unit: mm
tile_step_asi = (tile_axis_ROI.*pixel_size_um) * (1-tile_axis_overlap) * 10.0 #unit: 1/10 um


# height axis setup
# this is more complicated, since we have an oblique light sheet
# the height of the scan is the length of the ROI in the tilted direction * sin(tilt angle)
# however, it may be better to hardcode displacement based on measurements of the light sheet Rayleigh length
# for now, go with overlap calculation
height_axis_overlap=0.2
height_axis_range_asi = np.abs(height_axis_end_asi-height_axis_start_asi)
height_axis_ROI = ROI[4]-ROI[1]*pixel_size #unit: um
height_axis_step_um = (height_axis_ROI*pixel_size_um*np.sin(30.*(np.pi/180.)))*(1-height_axis_overlap) # unit: um
height_axis_step_mm = (height_axis_ROI*pixel_size_um*np.sin(30.*(np.pi/180.)))*(1-height_axis_overlap) * .001 #unit: mm
height_axis_step_asi = (height_axis_ROI*pixel_size_um*np.sin(30.*(np.pi/180.)))*(1-height_axis_overlap) * 10.0 #unit: 1/10 um



### Setup ASI Tiger controller

In [ ]:
stage_port = 'COM4'

# Setup PLC card to give start trigger
plcName = 'PLogic:E:36'
propPosition = 'PointerPosition'
propCellConfig = 'EditCellConfig'
addrOutputBNC3 = 35
addrStageSync = 46  # TTL5 on Tiger backplane = stage sync signal
 
# connect stage sync signal to BNC output
core.set_property(plcName, propPosition, addrOutputBNC3)
core.set_property(plcName, propCellConfig, addrStageSync)

# set tile axis speed for all moves
command = 'SPEED Y=.5'
answer = send_command_Tiger(stage_port,command,False)

# move tile axis to initial position
# expects 1/10 um
command = 'MOVE Y='+start_tile_axis_asi
answer = send_command_Tiger(stage_port,command,True);

# set scan axis speed for large move to initial position
command = 'SPEED X=.5'
answer = send_command_Tiger(stage_port,command,False)

# move scan scan stage to initial position
# expects 1/10 um
command = 'MOVE X='+start_scan_axis_asi
answer = send_command_Tiger(stage_port,command,True);

# set scan axis speed to correct speed for continuous stage scan
# expects mm/s
command = 'SPEED X='+scan_rate
answer = send_command_Tiger(stage_port,command,False)

# set scan axis to true 1D scan with no backlash
command = '1SCAN X? Y=0 Z=9 F=0'
answer = send_command_Tiger(stage_port,command,False)

# set range for scan axis
# expects mm
command = '1SCANR X='+start_scan_axis_mm+' Y='+end_scan_axis_mm+' R=50'
answer = send_command_Tiger(stage_port,command,False)

### Setup BSI Express camera

In [5]:
# set exposure
core.set_exposure(exposure_ms)

# set camera up for stage scan
core.set_config('Camera_Setup','CMS_stage_scan')

# set camera into low noise readout mode
core.set_config('CameraReadout','2-CMS')
core.wait_for_config('CameraReadout','2-CMS')

# set camera fan to high and temp to cold
core.set_config('CameraTemp','Cold')
core.wait_for_config('CameraTemp','Cold')

# set camera to trigger first mode
core.set_config('CameraTrigger','Trigger first')
core.wait_for_config('CameraTrigger','Trigger first')

# set camera to send exposure out on TRIG1
core.set_config('OutputTrigger','Global Exposure')
core.wait_for_config('OutputTrigger','Global Exposure')

# crop FOV
mmc.set_roi(*ROI)

### Setup Coherent OBIS laser box

In [ ]:
# set all lasers to digital external triggering. 
# Galvo controller takes lower bandwdith digital triggering of 561 (~18 microsecond response) into account

core.set_config('Obis-State-405','Off')
core.wait_for_config('Obis-State-405','Off')
core.set_config('Obis-State-405','External_digital')
core.wait_for_config('Obis-State-405','External_digital')
core.set_property('Coherent-Scientific Remote','Laser 405-100C - PowerSetpoint (%)',channel_powers[0])

core.set_config('Obis-State-488','Off')
core.wait_for_config('Obis-State-488','Off')
core.set_config('Obis-State-488','External_digital')
core.wait_for_config('Obis-State-488','External_digital')
core.set_property('Coherent-Scientific Remote','Laser 488-150C - PowerSetpoint (%)',channel_powers[1])

core.set_config('Obis-State-561','Off')
core.wait_for_config('Obis-State-561','Off')
core.set_config('Obis-State-561','External_digital')
core.wait_for_config('Obis-State-561','External_digital')
core.set_property('Coherent-Scientific Remote','Laser OBIS LS 561-150 - PowerSetpoint (%)',channel_powers[2])

core.set_config('Obis-State-637','Off')
core.wait_for_config('Obis-State-637','Off')
core.set_config('Obis-State-637','External_digital')
core.wait_for_config('Obis-State-637','External_digital')
core.set_property('Coherent-Scientific Remote','Laser 637-140C - PowerSetpoint (%)',channel_powers[3])

core.set_config('Obis-State-730','Off')
core.wait_for_config('Obis-State-730','Off')
core.set_config('Obis-State-730','External_digital')
core.wait_for_config('Obis-State-730','External_digital')
core.set_property('Coherent-Scientific Remote','Laser 730-30C - PowerSetpoint (%)',channel_powers[4])

### Setup homebuilt galvo/laser controller
Our galvo/laser controller is a [Teensy 3.5](https://www.pjrc.com/teensy/) with a [12-bit bi-phasic DAC](https://www.tindie.com/products/visgence/power-dac-module/). The PCB design and controller software are found in this repo. The controller performs the following,
1. Waits for camera exposure out signal. This signal is sent when all rows are exposing
2. Activates laser while galvo sits just outside FOV
3. Sweeps galvo across the FOV, covering the entire FOV once during exposure time. The 12-bit DAC board we use requires 4.5 us to achieve a full change, so we calculate the number of possible lines using this. In reality, the unit performs better  than this for small changes, but we get sufficient resolution with this method even for 1 ms exposures.
4. Blanks laser at end of sweep, which is also just outside FOV.
5. Returns galvo to initial position once laser is blanked.

We cannot sweep the mirror back and forth, because there is a slight skew to the data due to the combination of the galvo sweep + constant stage scan. At slow stage speeds with ca. <2 pixel displacement between images, this is not visible. But, as the stage speed increases, we have to correct for this skew in the post processing.

This approach eliminates motion blur and limits shadowing. Inspiration for this approach was taken from various papers, especially [IsoView](https://doi.org/10.1038/nmeth.3632) by the Keller lab and our previous work on [interia-free light sheet](https://doi.org/10.1038/s41467-017-00514-7). The galvo/laser controller used here is a modified version of the custom DAC solution used in our previous works. Because the Coherent OBIS laser box has 50 Ohm inputs, additional circuitry is needed to drive the digital lines using the 3.3V Teensy.

In [146]:
# port for Teensy
Teensy_port = 'COM5'

# settings for galvo sweep (determined independently)
galvo_min_V = -2.0 #unit: V
galvo_max_V = +2.0 #unit: V

# bi-phasic DAC external power values
supply_min_V = -3.3 #unit: V
supply_max_V = +3.3 #unit: V

# calculate min and max DAC value in 12-bit space
min_12bit_value = np.rint((galvo_min_V)*(2**12-0)/(supply_max_V-supply_min_V) + 2048).astype(int)
max_12bit_value = np.rint((galvo_max_V)*(2**12-0)/(supply_max_V-supply_min_V) + 2048).astype(int)

# determine number of lines to skip to make sure DAC does not have instability
DAC_settling_time=0.0045 #unit: ms
native_sweep_time = (max_12bit_value-min_12bit_value)*DAC_settling_time #unit: ms
factor = np.rint(native_sweep_time / exposure_ms).astype(int)
if factor == 0:
    factor = 1

# explicit calculation of DAC values used to perform sweep is independently performed on the galvo/laser controller.
# Replicated here for diagnosis.
dac_array = np.rint(np.arange(min_12bit_value,max_12bit_value,factor)).astype(int)

## Run acquisition
We don't use the built-in multi-dimensional acquisition. This is because,
1. We acquire events as the "z" position within pycromanager, but these events are actually positions of the scan axis (x).
2. The acquisition during any given tile scan {1 fluidics round (r), 1 height axis position (zstage) , 1 tile axis position (y) , 1 channel (c), N scan axis images (x)} is completely hardware triggered. All pycromanager does is acquire each image and store it to a SSD array as fast as possible.
3. We may change the height axis position (zstage) independent of the "z" acquisition parameter given by the scan axis (x) to image samples larger than the Rayleigh length of the oblique light sheet.
4. We may change the exposure time between different channels, which changes the scan axis (x) and galvo/laser controller (c) parameters.
5. We need to make sure the tile axis (y) has returned to initial position before starting again.
6. Our ASI Tiger controller (x,y,zstage) often exhibits serial communication problems. So we make sure that the command is received and executed before continuing.
7. We use a custom galvo/laser controller (c) that does not fit into the definition of Micromanager 2.0 sequenceable hardware. Our approach eliminates motion blur during a stage scan and reduces shadowing.
8. Our Numba deskewing and BDV H5 creation code requires each strip scan {1 fluidics round (r), 1 height axis position (zstage) , 1 tile axis position (y) , 1 channel (c), N scan axis images (x)} to be in it's own file and/or directory.
9. We may run an external fluidics controller (r) after a complete volume scan for iterative rounds of labeling.

In [12]:
if __name__ == '__main__':

    # create events to hold all of the scan axis images during constant speed stage scan
    # we call this 'z' here, even though it is actually oblique images acquired by moving scan axis (x) in our system
    events = []
    for z in range(number_of_images):
        event.append({'axes': {'z': z}})

    # loop through all tile axis positions
    for y in length(tile_axis_positions_asi):
        # move tile axis to new position
        command = 'MOV Y='+tile_axis_positions_asi[y]
        answer = send_Tiger_command(stage_port, command, True)
        
        # grab actual tile axis stage position for metadata file
        
        # loop through all height axis positions at current tile axis position
        for zstage in length(height_axis_positions_asi):
        # move height axis to new position
        command = 'MOV M='+height_axis_positions_asi[zstage]
        answer = send_Tiger_command(stage_port, command, True)
        
        # grab actual height axis position for metadata file
        
            # loop through all channels at current height axis and tile axis positions
            for c in length(channel_states):
                if channel_states[c] == 1:
                    # set galvo controller to trigger current channel
                    setup_galvo(galvo_port,min_12bit_value,max_12bit_value,exposure,channel,number_of_images)

                    # setup file name
                    save_name = 'scan_'+'y_'+y+'z_'+zstage+'c_'+c
                    # run acquisition
                    # TO DO: properly handle an error here if camera driver fails to return expected number of images.
                    with Acquisition(directory=save_directory,name=save_name,post_camera_hook_fn=start_stage) as acq:
                        acq.acquire(events)
                        acq.await_completion()

                    # ensure scan axis has returned to initial position
                    command = 'STATUS';
                    answer = send_Tiger_command(stage_port, command, True)

        #-----------------------------------------------------------------------------------------------------------
        # cycle camera mode after one tile to avoid issues with Photometrics driver not returning expected
        # number of images. This always occurs after a acquiring large number of images in a single run (n~5 million)
        # STATUS OF BUG: 2020.10.04 - Photometrics acknowledges this is a problem that we encounter and they verify.
        #                             No solution offered.

        # set camera into HDR readout and no trigger mode
        core.set_config('CameraSetup','HDR_free_run')
        core.wait_for_config('CameraSetup','HDR_free_run')
        sleep(100)

        # set camera to CMS readout and trigger first mode
        core.set_config('CameraSetup','CMS_stage_scan')
        core.wait_for_config('CameraSetup','CMS_stage_scan')
        sleep(100)
        #-----------------------------------------------------------------------------------------------------------
        
        
# set camera into HDR readout and no trigger mode
core.set_config('CameraSetup','HDR_free_run')
core.wait_for_config('CameraSetup','HDR_free_run')

NameError: name 'number_of_images' is not defined